In [1]:
from model_dict import *
from inference import *
import pandas as pd
import random, csv
import guidance
from transformers import AutoTokenizer, pipeline, logging

/opt/conda/envs/Eval4NLP23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Instantiate guidance

In [2]:
summ_path = '../data/summarization/train_summarization.tsv'
summ_df = pd.read_csv(summ_path, delimiter="\t", quoting=csv.QUOTE_NONE)

mt_path = '../data/en_de/train_en_de.tsv'
mt_df = pd.read_csv(mt_path, delimiter="\t", quoting=csv.QUOTE_NONE)

In [3]:
model_name = 'WizardLM-13B-V1.1-GPTQ'
model_path = f'../models/{model_name}'

model, tokenizer, u_prompt, a_prompt = load_from_catalogue(model_name, model_path)

2023-08-23 07:01:35,571 - 140558343946624 - qlinear_cuda_old.py-qlinear_cuda_old:15 - WARNING: CUDA extension not installed.
2023-08-23 07:01:44,234 - 140558343946624 - modeling.py-modeling:1093 - WARNING: The safetensors archive passed at ../models/WizardLM-13B-V1.1-GPTQ/wizardlm-13b-v1.1-GPTQ-4bit-128g.no-act.order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
2023-08-23 07:01:51,829 - 140558343946624 - fused_llama_mlp.py-fused_llama_mlp:280 - WARNING: skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [4]:
g_model = guidance.llms.Transformers(
            model, tokenizer=tokenizer, trust_remote_code=True,
)
guidance.llms.Transformers.cache.clear()
guidance.llm = g_model

In [6]:
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     temperature=0.7,
#     top_p=0.95,
#     repetition_penalty=1.15
# )

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

#### Zero-Shot

In [14]:
input_prompt = '''
{{prompt_placeholder}}
Score the summarization with respect to the summarized document\
on a continuous scale from 0 to 100, where a score of zero means\
irrelevant, factually incorrect and not readable" and score of one hundred means\
relevant, factually correct, good readability.
----
Source text: {{gt}}
Summary: {{hyp}}
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [16]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
zero_shot = structure_program(
    gt=summ_df['SRC'][2],
    hyp=summ_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt
)
torch.cuda.empty_cache()

zero_shot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the summarization with respect to the summarized documenton a continuous scale from 0 to 100, where a score of zero meansirrelevant, factually incorrect and not readable" and score of one hundred meansrelevant, factually correct, good readability.
---- 
Source text: "(CNN)A fiery sunset greeted people in Washington Sunday. The deep reddish color caught Seattle native Tim Durkan's eye. He photographed a handful of aerial shots of the sunset warming the city's skyline and shared them on CNN iReport. The stunning sunsets were the result of raging wildfires in parts of Siberia. ""The dramatic sunsets began showing up over the weekend and had Seattle locals wondering where the amber-colored haze was originating from,"" Durken said. The fires were started in southeastern Siberia, by farmers burning grass in their fields. But on April 14, it is believed that the flames quickly grew out of control because of strong winds and spread throughout the region, according to CNN affiliate KOMO-TV. As a result, the fires have destroyed dozens of villages in the region. Rescue crews were able to put out the flames. However, the lingering smoke from the widespread fires were picked up by atmospheric winds. The winds carried the smoke from Siberia across the Pacific Ocean and brought it to the Pacific Northwest. Parts of Oregon, Washington and British Columbia are seeing the results of the smoke, wind and solar light combination. The reason people are seeing an intense red sunset is a result of smoke particles filtering out the shorter wavelength colors from the sunlight like greens, blues, yellows and purples, KOMO-TV said. That means colors like red and orange are able to penetrate the air unfiltered. The colors are especially intense during sunrises and sunsets because there is more atmosphere for the light to travel through to get to a person's eye. As the smoke starts to dissipate, air quality will get better and these fiery sunsets will lose their reddish hue." 
Summary: tim durkan photographed aerial shots of the sunset warming the city 's skyline and shared them on cnn ireport . the fires were started in southeastern siberia , by farmers burning grass in their fields . the flames quickly grew out of control because of strong winds and spread throughout the region . 
 Assistant: 
Score: 85

In [18]:
input_prompt = '''
{{prompt_placeholder}}
Score the following translation from {{source_lang}} to {{target_lang}} with respect to\
the source sentence on a continuous scale from 0 to 100, where a score of zero means\
"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
----
{{source_lang}} source: "{{gt}}"
{{target_lang}} translation: "{{hyp}}"
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [19]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
zero_shot = structure_program(
    gt=mt_df['SRC'][2],
    hyp=mt_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt,
    source_lang="English",
    target_lang="German"
)
torch.cuda.empty_cache()

zero_shot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the following translation from English to German with respect tothe source sentence on a continuous scale from 0 to 100, where a score of zero means"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
----
 English source: " International buyers please note: our large photographs are sent rolled in postal tubes. "
 German translation: " Internationale Käufer beachten Sie bitte: Unsere großen Fotos werden in Postrohren gerollt geschickt. "
 Assistant: 
Score: 95

#### Few-Shot

In [20]:
examples = [
    {'gt': f'{summ_df["SRC"][0]}',
    'hyp': f'{summ_df["HYP"][0]}',
    'score': f'{summ_df["Score"][0]}'},
    {'gt': f'{summ_df["SRC"][1]}',
    'hyp': f'{summ_df["HYP"][1]}',
    'score': f'{summ_df["Score"][1]}'},
]

input_prompt = '''
{{prompt_placeholder}}
Score the summarization with respect to the summarized document\
on a continuous scale from 0 to 100, where a score of zero means\
irrelevant, factually incorrect and not readable" and score of one hundred means\
relevant, factually correct, good readability.
----
{{~! display the few-shot examples ~}}
{{~#each examples}}
Source text: {{this.gt}}
Summary: {{this.hyp}}
{{response_placeholder}}
Score: {{this.score}}
----
{{~/each}}

{{~! place the real question at the end }}
Source text: {{gt}}
Summary: {{hyp}}
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [21]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
few_shot = structure_program(
    examples=examples,
    gt=summ_df['SRC'][2],
    hyp=summ_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt
)
torch.cuda.empty_cache()

few_shot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the summarization with respect to the summarized documenton a continuous scale from 0 to 100, where a score of zero meansirrelevant, factually incorrect and not readable" and score of one hundred meansrelevant, factually correct, good readability.
---- 
Source text: These extraordinary images show how an enormous jumbo jet has been converted into a luxury home for a mystery billionaire - complete with bedrooms, multiple lounges and an onboard restaurant. The custom-built Boeing 747 is believed to have cost its ultra-wealthy owner £400million after it was customised to fit his specifications exactly over a three-year period. The jumbo jet would normally carry as many as 600 passengers - but this version was built for just a single tycoon, although he will be able to fit dozens of guests. It contains a large dining room where travellers can enjoy meals cooked in an adjoining kitchen, with a lounge and 'state room' at the back of the plane. More sleeping space is provided in the 'aeroloft' on the top deck of the vessel, with eight full beds for passengers who prefer to get some shut-eye on flights. Luxury: This is the lounge on board a custom-built Boeing 747-8, as depicted in an artist's impression by Greenpoint Technologies Shuteye: The master bedroom of the £400million plane has a huge bed as well as an armchair and sofa for when the owner needs to relax Business: The conference room of the plane could be perfect for meetings in the sky with its long table in the centre Chill-out: The lounge decor looks like a luxury hotel, belying its true location inside a plane similar to those used by commercial airlines Executive: This room, which can be used as an office or a guest bedroom, gives a hint to its true location thanks to the porthole-style windows Kicking back: The main suite could easily be mistaken for a room in a top hotel thanks to its over-the-top decorations Restaurant: The central portion of the plane hosts a dining room which is attached to a kitchen, allowing the owner to make use of the services of a private chef Shiny: The jumbo jet is one of the most luxurious private planes ever built, with the price tag believed to have reached a total of £400million Sofa: Parts of the plane are set aside for guests to relax while watching television, as the vessel can carry several dozen passengers Plush: Another artist's impression of the main state room, which also contains a desk so the owner can switch between work and relaxation Welcoming: Images of the lounge show a space very different to the normal aeroplane cabin, with the traditional rows of seats completely discarded in favour of a hotel-like ambience Exclusive: Few people can ever imagine flying in this sort of luxury - far more glitzy than first class on commercial flights Plan: The layout of the Boeing 747 is shown in this plan published by Greenpoint Technologies Gigantic: The luxury interior has been built inside a Boeing 747-8 similar to this one, which can hold up to 600 passengers 
Summary: the custom-built boeing 747 is believed to have cost its ultra-wealthy owner # 400million after it was customised to fit his specifications exactly over a three-year period . the jumbo jet would normally carry as many as 600 passengers - but this version was built for just a single tycoon , although he will be able to fit dozens of guests . it contains a large dining room where travellers can enjoy meals cooked in an adjoining kitchen , with a lounge and ` state room ' at the back of the plane . 
 Assistant: 
Score: 4.583333333325 
----
Source text: One Missouri mom is making the best out of a less than ideal situation. Becky Schoenig of St. Louis, Missouri was shocked when she realized her brand new 2015 Ford Fusion had been stolen from her driveway on Monday night, just a little over a mo

In [24]:
examples = [
    {'gt': f'{mt_df["SRC"][0]}',
    'hyp': f'{mt_df["HYP"][0]}',
    'score': f'{mt_df["mqm"][0]}'},
    {'gt': f'{mt_df["SRC"][1]}',
    'hyp': f'{mt_df["HYP"][1]}',
    'score': f'{mt_df["mqm"][1]}'},
]

input_prompt = '''
{{prompt_placeholder}}
Score the following translation from {{source_lang}} to {{target_lang}} with respect to\
the source sentence on a continuous scale from 0 to 100, where a score of zero means\
"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
----
{{~! display the few-shot examples ~}}
{{~#each examples}}
{{source_lang}} source: {{this.gt}}
{{target_lang}} translation: {{this.hyp}}
{{response_placeholder}}
Score: {{this.score}}
----
{{~/each}}

{{~! place the real question at the end }}
{{source_lang}} source: {{gt}}
{{target_lang}} translation: {{hyp}}
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [25]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
few_shot = structure_program(
    examples=examples,
    gt=mt_df['SRC'][2],
    hyp=mt_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt,
    source_lang="English",
    target_lang="German"
)
torch.cuda.empty_cache()

few_shot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the following translation from English to German with respect tothe source sentence on a continuous scale from 0 to 100, where a score of zero means"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
---- 
 English source: Then Dominic Cummings, once Johnson's closest adviser, promised pictures had been taken of parties, and so with bated breath critics of the government waited for them to surface. 
 German translation: Dann versprach Dominic Cummings, einst Johnsons engster Berater, dass Fotos von Partys gemacht worden seien, und so warteten die Kritiker der Regierung mit angehaltenem Atem darauf, dass sie auftauchten. 
 Assistant: 
Score: -0.0 
----
 English source: Norton has been outperforming the competition in many reputable head-to-head tests, and only Norton has won the PC Magazine Editors’ Choice Award 34 times, including 11 years in a row – more than any other security company. 
 German translation: Norton konnte die Konkurrenz in vielen seriösen Vergleichstests übertreffen und nur Norton hat 34 Mal, darunter elf Jahre in Folge, den Editors' Choice Award des PC Magazine gewonnen, mehr als irgendein anderes Sicherheitsunternehmen. 
 Assistant: 
Score: -0.0 
---- 
 English source: International buyers please note: our large photographs are sent rolled in postal tubes. 
 German translation: Internationale Käufer beachten Sie bitte: Unsere großen Fotos werden in Postrohren gerollt geschickt. 
 Assistant: 
Score: 95

#### Chain of Thought

In [26]:
input_prompt = '''
{{prompt_placeholder}}
Score the summarization with respect to the summarized document\
on a continuous scale from 0 to 100, where a score of zero means\
irrelevant, factually incorrect and not readable" and score of one hundred means\
relevant, factually correct, good readability.
    
Evaluation Steps:
1. Read the summary carefully and identify the main topic and key points.
2. Read the summary and compare it to the source text.\
Check if the summary covers the main topic and key points of source text,\
and if it presents them in a clear and logical order
3. Assign a score.
----
{{~! place the real question at the end }}
Source text: {{gt}}
Summary: {{hyp}}
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [27]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
cot = structure_program(
    gt=summ_df['SRC'][2],
    hyp=summ_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt
)
torch.cuda.empty_cache()

cot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the summarization with respect to the summarized documenton a continuous scale from 0 to 100, where a score of zero meansirrelevant, factually incorrect and not readable" and score of one hundred meansrelevant, factually correct, good readability.
 
Evaluation Steps:
1. Read the summary carefully and identify the main topic and key points.
2. Read the summary and compare it to the source text.Check if the summary covers the main topic and key points of source text,and if it presents them in a clear and logical order
3. Assign a score.
----
Source text: "(CNN)A fiery sunset greeted people in Washington Sunday. The deep reddish color caught Seattle native Tim Durkan's eye. He photographed a handful of aerial shots of the sunset warming the city's skyline and shared them on CNN iReport. The stunning sunsets were the result of raging wildfires in parts of Siberia. ""The dramatic sunsets began showing up over the weekend and had Seattle locals wondering where the amber-colored haze was originating from,"" Durken said. The fires were started in southeastern Siberia, by farmers burning grass in their fields. But on April 14, it is believed that the flames quickly grew out of control because of strong winds and spread throughout the region, according to CNN affiliate KOMO-TV. As a result, the fires have destroyed dozens of villages in the region. Rescue crews were able to put out the flames. However, the lingering smoke from the widespread fires were picked up by atmospheric winds. The winds carried the smoke from Siberia across the Pacific Ocean and brought it to the Pacific Northwest. Parts of Oregon, Washington and British Columbia are seeing the results of the smoke, wind and solar light combination. The reason people are seeing an intense red sunset is a result of smoke particles filtering out the shorter wavelength colors from the sunlight like greens, blues, yellows and purples, KOMO-TV said. That means colors like red and orange are able to penetrate the air unfiltered. The colors are especially intense during sunrises and sunsets because there is more atmosphere for the light to travel through to get to a person's eye. As the smoke starts to dissipate, air quality will get better and these fiery sunsets will lose their reddish hue." 
Summary: tim durkan photographed aerial shots of the sunset warming the city 's skyline and shared them on cnn ireport . the fires were started in southeastern siberia , by farmers burning grass in their fields . the flames quickly grew out of control because of strong winds and spread throughout the region . 
 Assistant: 
Score: 60

In [31]:
input_prompt = '''
{{prompt_placeholder}}
Score the following translation from {{source_lang}} to {{target_lang}} with respect to\
the source sentence on a continuous scale from 0 to 100, where a score of zero means\
"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
    
Evaluation Steps:
1. Read source and translation independently.
2. Read translation and compare it to the source.\
 Check if the translation covers all the information and fluency of source,\
and if it doesn't have any syntatic errors.
3. Assign a score.
----
{{source_lang}} source: "{{gt}}"
{{target_lang}} translation: "{{hyp}}"
{{response_placeholder}}
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}
'''

In [30]:
structure_program = guidance(input_prompt, llm=g_model, caching=False)
cot = structure_program(
    gt=mt_df['SRC'][2],
    hyp=mt_df['HYP'][2],
    prompt_placeholder=u_prompt,
    response_placeholder=a_prompt,
    source_lang="English",
    target_lang="German"
)
torch.cuda.empty_cache()

cot()

Stop program 
 A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

 USER: 
Score the following translation from English to German with respect tothe source sentence on a continuous scale from 0 to 100, where a score of zero means"no meaning preserved" and score of one hundred means "perfect meaning and grammar".
 
Evaluation Steps:
1. Read source and translation independently.
2. Read translation and compare it to the source.Check if the translation covers all the information and fluency of source,and if it doesn't have any syntatic errors.
3. Assign a score.
----
 English source: " International buyers please note: our large photographs are sent rolled in postal tubes. "
 German translation: " Internationale Käufer beachten Sie bitte: Unsere großen Fotos werden in Postrohren gerollt geschickt. "
 Assistant: 
Score: 95

#### Fine-Grained

#### Generartion Probability

#### Majority Vote

#### Self-Refinement